In [0]:
#!pip install --upgrade tables
#!pip install eli5
#!pip install xgboost
#!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [6]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if type(df[feat][0]) is list: continue
  if SUFFIX_CAT in feat: continue
  df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc_num'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split('KM')[0].replace(' ','')) )
df['param_pojemność-skokowa-num'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))
df['param_przebieg_num'] = df['param_przebieg'].map(lambda x: -1 if str(x) == 'None' else int(x.split('km')[0].replace(' ','')))

In [0]:
xgb_params = {'max_depth' : 5,
              'n_estimators': 50,
              'learning_rate': 0.1,
              'seed': 0
              }

model = xgb.XGBRegressor(**xgb_params)

feats = ['param_napęd__cat', 'param_rok-produkcji','param_stan__cat', 'param_skrzynia-biegów__cat', 'param_faktura-vat__cat',
'param_moc_num', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat',
'param_pojemność-skokowa-num', 'seller_name__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat',
'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat', 'seller_type__cat','param_bezwypadkowy__cat','param_uszkodzony__cat','param_przebieg_num',
'param_zarejestrowany-w-polsce__cat','param_kraj-pochodzenia__cat']

#run_model(model, feats)

In [0]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK }

In [0]:
# space
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05 )),
    'max_depth':        hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror', 
    'n_estimators':     100,
    'seed':             0,
}

In [14]:
# run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.2, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.25, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.25, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_dep

{'colsample_bytree': 0.75,
 'learning_rate': 2,
 'max_depth': 7,
 'subsample': 0.9500000000000001}